# Quick and Dirty Diffusor Calibration

calibrates $gDT$ where $T$ is the tap point matrix, $D$ is the diffusor kernels (in this case, with no edges cut) and $g$ is the set of neuron gains.

This is meant to be used to drop into the existing numerical simulations of diffusor spread.

To make this more practical (working just a single pool), you would need to collect different sets with different diffusor cut conditions. To be complete, for each tap point location, you'd have kernels for (num DAC spreads) * {no cuts nearby, cut above, cut right, cut left, cut down, cut above+right, cut above+left, etc.}. For broad spreads, you'd have to add cuts 2 away, 3 away, etc.

In [ ]:
%load_ext autoreload
%autoreload 2

from pystorm.hal import HAL
from pystorm.PyDriver import bddriver as bd
from pystorm.hal.net_builder import NetBuilder
from pystorm.hal.calibrator import Calibrator, PoolSpec

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np

In [ ]:
# full chip
Y = X = 64
LY = LX = 0
SY = Y // 2
SX = X // 2
D = 1

DACS = dict(DAC_DIFF_G = 1024,
            DAC_DIFF_R = 600,
            DAC_SOMA_REF = 1024,
            DAC_SOMA_OFFSET = 2)

# need O(grid_space**2 samples)
# how many synapses to leave off between active synapses
SYN_GRID_SPACE = 8
assert(SYN_GRID_SPACE % 2 == 0) # use an even number to keep #taps even

KY = KX = SYN_GRID_SPACE * 2

# estimate_encoders for 1D takes 9 samples/trial 2 times, 1 s per sample
trial_extra = 33
print('runtime estimate:', (9 * 1 * 2 + trial_extra) * SYN_GRID_SPACE**2 / 60, 'minutes')


In [ ]:
hal = HAL()
cal = Calibrator(hal)

import time
t0 = time.time()
all_encs = {}
#for y_grid_idx in range(1):
#    for x_grid_idx in range(1):
for y_grid_idx in range(SYN_GRID_SPACE):
    for x_grid_idx in range(SYN_GRID_SPACE):
        print("RUNNING Y:", y_grid_idx, "X:", x_grid_idx)
        print("=======================")
        print((time.time() - t0) / 60, 'minutes elapsed')
        
        syn_TPM = np.zeros((SY, SX))
        syn_TPM[y_grid_idx::SYN_GRID_SPACE, x_grid_idx::SYN_GRID_SPACE] = 1
        TPM = NetBuilder.syn_taps_to_nrn_taps(syn_TPM.reshape((SY, SX, 1)))

        # use bias 3, want lots of spiking
        ps = PoolSpec(YX=(Y,X), loc_yx=(LY, LX), D=D, TPM=TPM, biases=3)
        ps.fmax = cal.optimize_fmax(ps, safety_margin=.95)
        
        # estimate encoders
        encs, offs, std_encs, std_offs, _, _ = \
            cal.get_encoders_and_offsets(ps, dacs=DACS, num_sample_angles=3, bin_time=2, num_bootstraps=20)
            
        all_encs[(y_grid_idx, x_grid_idx)] = dict(ps=ps, encs=encs, std_encs=std_encs)
        
import pickle
pck_fname = 'calibrate_diffusor_' + str(DACS['DAC_DIFF_G']) + '_' + str(DACS['DAC_DIFF_R']) + '.pck'
pickle.dump(all_encs, open(pck_fname, 'wb'))

print((time.time() - t0) / 60, 'minutes elapsed')

In [ ]:
import pickle
DAC_DIFF_G = DACS['DAC_DIFF_G']
DAC_DIFF_R = DACS['DAC_DIFF_R']
#DAC_DIFF_G = 1024
#DAC_DIFF_R = 600
pck_fname = 'calibrate_diffusor_' + str(DAC_DIFF_G) + '_' + str(DAC_DIFF_R) + '.pck'
all_encs = pickle.load(open(pck_fname, 'rb'))

for (y_grid_idx, x_grid_idx), enc_dict in all_encs.items():
    encs = enc_dict['encs']
    std_encs = enc_dict['std_encs']
    ps = enc_dict['ps']
    
    # plot raw responses and errors
    ps_for_plot = ps.copy()
    ps_for_plot.TPM = np.hstack((ps.TPM, ps.TPM))
    Calibrator.plot_encs_yx(np.hstack((encs, std_encs)), ps_for_plot, figheight=4)
    


In [ ]:
plotlog=False

kernels = {}
Ksums = []
for (y_grid_idx, x_grid_idx), enc_dict in all_encs.items():
    encs = enc_dict['encs']
    std_encs = enc_dict['std_encs']
    
    # extract kernels
    yx_encs = np.zeros((Y + 2*SYN_GRID_SPACE, X + 2*SYN_GRID_SPACE))
    yx_std_encs = np.zeros_like(yx_encs)
    
    # zero-padded outside
    yx_encs[SYN_GRID_SPACE:-SYN_GRID_SPACE, SYN_GRID_SPACE:-SYN_GRID_SPACE] = encs.reshape((Y, X))
    yx_std_encs[SYN_GRID_SPACE:-SYN_GRID_SPACE, SYN_GRID_SPACE:-SYN_GRID_SPACE] = std_encs.reshape((Y, X))
    
    # nrn idxs
    for y_center_idx in range(y_grid_idx*2, Y, SYN_GRID_SPACE*2):
        for x_center_idx in range(x_grid_idx*2, X, SYN_GRID_SPACE*2):
            if (y_center_idx // 2) % 2 == 0:
                xshift = 0
            else:
                xshift = 1

            ymin = SYN_GRID_SPACE + y_center_idx - SYN_GRID_SPACE
            xmin = SYN_GRID_SPACE + x_center_idx - SYN_GRID_SPACE + xshift
            ymax = SYN_GRID_SPACE + y_center_idx + SYN_GRID_SPACE + 1
            xmax = SYN_GRID_SPACE + x_center_idx + SYN_GRID_SPACE + 1 + xshift
            
            K = yx_encs[ymin:ymax, xmin:xmax]
            Kpos = K.copy()
            #Kpos[Kpos < 0] = 0
            Kpos[np.isnan(Kpos)] = 0
            
            Kerr = yx_std_encs[ymin:ymax, xmin:xmax]
            
            # cancel out sketchy measurements
            # 95% confidence interval bigger than half measured value
            big_err = Kerr * 2 > Kpos * .5
            
            Kpos[big_err] = 0
            
            Ksums.append(np.sum(Kpos))
            
            kernels[y_center_idx // 2, x_center_idx // 2] = dict(K=Kpos, Kerr=Kerr)
            
print(np.mean(Ksums))
            
all_encs_flat = np.abs(all_encs[0,0]['encs'].flatten())
all_encs_flat[np.isnan(all_encs_flat)] = 0
vmin = 0
vmax = np.sort(all_encs_flat)[int(.99 * len(all_encs_flat))]

from mpl_toolkits.axes_grid1 import make_axes_locatable

pmin = 8
pmax = 16
PYX = pmax - pmin

fig, ax = plt.subplots(2*PYX, PYX, figsize=(15, 30))

for (sy, sx), k_dict in kernels.items():
    if sy >= pmin and sy < pmax and sx >= pmin and sx < pmax:
        spy = sy - pmin
        spx = sx - pmin
        
        Kpos = k_dict['K']
        Kerr = k_dict['Kerr']
        
        if plotlog:
            im = ax[spy, spx].imshow(np.log(Kpos + 1), vmin=np.log(vmin + 1), vmax=np.log(vmax + 1))
            plt.colorbar(im)
            
        else:
            #im = ax[spy, spx].imshow(Kpos, vmin=vmin, vmax=vmax)
            this_ax = ax[2*spy, spx]
            im = this_ax.imshow(Kpos, vmin=vmin, vmax=vmax)
            divider = make_axes_locatable(this_ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            this_ax.axis('off')
            plt.colorbar(im, cax=cax)
            
            this_ax = ax[2*spy + 1, spx]
            im = this_ax.imshow(2 * Kerr, cmap='gray_r') # ~95% confidence
            divider = make_axes_locatable(this_ax)
            cax = divider.append_axes('right', size='5%', pad=0.05)
            this_ax.axis('off')
            plt.colorbar(im, cax=cax)
plt.tight_layout(w_pad=.02, h_pad=.02, pad=.01)

In [ ]:
import sys
sys.stdout.write('\a')
sys.stdout.flush()